This notebook uses the RBIntel data to create a portion of BNIA'S Vital Signs report

###**Indicators from this csv**


*   dom - Median Number of Days on the Market
*   cashsa - Percentage of residential sales for cash
*   reosa - Percentage of residential sales in foreclosure (REO)



In [ ]:
#export
#import needed libraries
import pandas as pd

In [ ]:
#export
#Read data
RBIntel_2021 = pd.read_csv('RBIntel_2021_BaltRegion_CSA_City.csv', encoding="cp850") #utf8 encoding kept giving an error.

#The crosswalk table is used to append CSA2010 names. 
crosswalk = pd.read_csv('https://raw.githubusercontent.com/BNIA/VitalSigns/main/CSA_2010_and_2020.csv')

#Keep necessary columns from crosswalk table
crosswalk = crosswalk[['CSA2010', 'CSA2020']]

#Remove duplicate rows.
crosswalk = crosswalk.drop_duplicates()

#Remove rows where Baltimore City column is empty
RBIntel_2021 = RBIntel_2021[RBIntel_2021['BaltCity'] == 'Baltimore City']

####dom Indicator

In [ ]:
#export
#Keep only necessary columns
dom = RBIntel_2021[["DaysonMark", "CSA2020"]]

#Remove rows (homes) that have been in the market for 0 days.
dom = dom[dom['DaysonMark'] > 0]

#Add Baltimore City row. Calculate its Median.
dom.loc[len(dom.index)] = [dom['DaysonMark'].median(), "Baltimore City"]

#Group by CSA
dom = dom.groupby(['CSA2020']).median()

#make index the CSA2020 column
dom.reset_index(inplace=True)

#Append CSA2010 names
dom = dom.merge(crosswalk, on='CSA2020', how='inner')

#Rename indicator Column
dom = dom.rename(columns={'DaysonMark':'domXX'})

#Change column order to clean things up.
dom = dom[['CSA2010', 'CSA2020', 'domXX']]

#Sort rows alphabetically 
dom = dom.sort_values('CSA2010')

####cashsa Indicator

In [ ]:
#export
#Keep only necessary columns
cashsa = RBIntel_2021[["BuyerFinan", "CSA2020"]]

#Numerator
cashsa['count'] = 1

#Denominator
denom = cashsa.copy()

#Only keep records where BuyerFinan has the word "Cash"
cashsa = cashsa[cashsa['BuyerFinan'].str.contains('Cash', regex=True, na=False)]

#Add Baltimore City row to both Numerator and Denominator
cashsa.loc[len(cashsa.index)] = ["Cash", "Baltimore City", cashsa['count'].sum()]
denom.loc[len(denom.index)] = ["Cash", "Baltimore City", denom['count'].sum()]

#Group by CSA
cashsa = cashsa.groupby(['CSA2020']).sum()
denom = denom.groupby(['CSA2020']).sum()

#Merge both tables
cashsa = cashsa.merge(denom, on='CSA2020', how='inner')

#Calculate indicator
cashsa['cashsaXX'] = (cashsa['count_x'] / cashsa['count_y']) *100

#make index the CSA2020 column
cashsa.reset_index(inplace=True)

#Append CSA2010 names
cashsa = cashsa.merge(crosswalk, on='CSA2020', how='inner')

#Change column order to clean things up.
cashsa = cashsa[['CSA2010', 'CSA2020', 'count_x', 'count_y' ,'cashsaXX']]

#Sort rows alphabetically 
cashsa = cashsa.sort_values('CSA2010')

####reosa Indicator

In [ ]:
#export
#Keep only necessary columns
reosa = RBIntel_2021[["Foreclosur", "CSA2020"]]

#Numerator
reosa['count'] = 1

#Denominator
denom = reosa.copy()

#Only keep records where Foreclosur = 'Y' (This column should only have 'Y's and 'N's)
reosa = reosa[reosa['Foreclosur'] == "Y"]

#Add Baltimore City row to both tables
reosa.loc[len(reosa.index)] = ["Y", "Baltimore City", reosa['count'].sum()]
denom.loc[len(denom.index)] = ["Y", "Baltimore City", denom['count'].sum()]

#Group by CSA
reosa = reosa.groupby(['CSA2020']).sum()
denom = denom.groupby(['CSA2020']).sum()

#Merge both tables
reosa = reosa.merge(denom, on='CSA2020', how='inner')

#make index the CSA2020 column
reosa.reset_index(inplace=True)

#Append CSA2010 names
reosa = reosa.merge(crosswalk, on='CSA2020', how='inner')

#Change column order to clean things up.
reosa = reosa[['CSA2010', 'CSA2020', 'count_x', 'count_y']]

#Calculate Indicator
reosa['reosaXX'] = (reosa['count_x'] / reosa['count_y']) *100

#Sort rows alphabetically 
reosa = reosa.sort_values('CSA2010')